# Exploiter le guide des recettes
Dans cet exercice, nous introduisons le Retrieval Augmented Generation (RAG), une approche qui permet à un agent basé sur un LLM de s’appuyer sur des données externes pour générer des réponses plus fiables, précises et à jour. Plutôt que de se reposer uniquement sur les connaissances internes du modèle, le LLM va d’abord rechercher des informations pertinentes dans une base de documents, puis utiliser ces informations comme contexte pour produire sa réponse.

Le RAG est particulièrement utile lorsque l’on travaille avec des documents spécifiques (documentation, articles, données métier) ou lorsque l’on souhaite éviter les hallucinations. Dans cet exercice, nous allons combiner DSPy pour l’orchestration du raisonnement et Qdrant comme base de données vectorielle pour la recherche sémantique.

## Définition des variables
Les variables sont lues depuis le fichier [.env](../../.env)

In [1]:
from dotenv import dotenv_values
config = dotenv_values("../../.env")

llm_model = config.get('LLM_MODEL')
api_key = config.get('LLM_API_KEY')
# Uncomment for local api call
# api_base = config.get('LLM_API_URL')


qdrant_url = config.get('QDRANT_URL')
qdrant_api_key = config.get('QDRANT_API_KEY')

## Configuration du llm sur dspy

In [2]:
import dspy
from dspy_qdrant import QdrantRM
from qdrant_client import QdrantClient
from langchain_huggingface import HuggingFaceEmbeddings

# Ton vectorizer (embedding) francophone
hf_embed = HuggingFaceEmbeddings(
    model_name="manu/bge-fr-en",  # modèle francophone + multilingue
    encode_kwargs={"normalize_embeddings": True}
)

def vectorizer(text_or_texts):
    if isinstance(text_or_texts, list):
        return hf_embed.embed_documents(text_or_texts)
    else:
        return hf_embed.embed_query(text_or_texts)


lm = dspy.LM(llm_model, api_key=api_key)
# Uncomment for local api call
#lm = dspy.LM(llm_model, api_base=api_base, track_usage=True, temperature=1.5, max_tokens=1024)

client = QdrantClient(qdrant_url, api_key=qdrant_api_key)
rm = QdrantRM(
    qdrant_collection_name="livre-recette", 
    qdrant_client=client,
    vector_name="recette",
    document_field="text",
    vectorizer=vectorizer
)


dspy.configure_cache(
    enable_disk_cache=False,
    enable_memory_cache=False,
)
dspy.configure(lm=lm, rm=rm)

## Créer le Retriever

Cette section définit la fonction `retriever` qui permet de rechercher les passages pertinents dans la base de données vectorielle. Elle utilise l'objet `dspy.Retrieve` pour effectuer une recherche sémantique basée sur la requête utilisateur et retourne un certain nombre de résultats (défini par le paramètre `k`).

In [3]:


def retriever(query: str, k: int) -> list[str]:
    retriever = dspy.Retrieve(k=k)
    revelant_passages = retriever(query).passages
    print(f"Revelant passages: {revelant_passages}")
    return revelant_passages

## Enrichir la génération

Cette section définit l'assistant qui utilise le raisonnement en chaîne (Chain of Thought) pour enrichir la réponse finale avec le contexte récupéré. Elle combine les passages pertinents de la base de données avec la question utilisateur dans un format spécifique pour produire une réponse plus précise et fondée sur des sources externes.

In [4]:

assistant = dspy.ChainOfThought("context, question -> response")

def rag(query: str, k: int) -> list[str]:
    revelant_passages = retriever(query, k)
    return assistant(context=revelant_passages, question=query)

## Exécuter le RAG

Cette section montre comment exécuter le système RAG complet. Elle appelle la fonction `rag` avec une requête utilisateur spécifique et un nombre de passages à récupérer (k=2). Le résultat est affiché à l'aide de la bibliothèque rich, permettant d'obtenir une sortie formatée et l'historique des appels du modèle pour analyse.

In [5]:
from rich import print
results = rag(query="Comment cuisiner un nachos garnis ?", k=2)

print(results)
print(lm.history)
dspy.inspect_history()

Revelant passages: ["title=LIVRE DE RECETTES POUR ETUDIANTS PAUVRES > NACHOS GARNIS entrees (A PARTAGER) > 
PREPARATION | text=1. Préchauffer le four à 400 ° F\n2. Sur une plaque de cuisson tapissée de papier-parchemin, 
étendre les croustilles de maïs.\n3. Ajouter les olives, les tomates fraiches et le poivron. Si désiré, le piment 
jalapeño et la salsa.\n4. Garnir de cheddar râpé.\n5. Cuire pendant environ 10 minutes ou jusqu'à ce que le fromage
ait fondu et qu'il soit légèrement doré.", "title=LIVRE DE RECETTES POUR ETUDIANTS PAUVRES > NACHOS GARNIS entrees 
(A PARTAGER) > INGREDIENTS | text=- 4 tasses de croustilles de maïs\n- 3 c. à table (45 ml) d'olives noires coupées
en tranches\n- 3 c. à table (45 ml) de tomates fraîches, épépinées et hachées (ou tomates en conserve)\n- 1 
tasse(250 ml) de cheddar râpé\n- 1/4 tasse (60 ml) de salsa (facultatif)\n- 1 c. à table (15 ml) de piment jalapeño
haché (facultatif)"]

Prediction(
    reasoning="Pour cuisiner un nachos garnis, il faut d'abord préchauffer le four à 400 °F. Ensuite, étaler les 
croustilles de maïs sur une plaque de cuisson recouverte de papier-parchemin. Ajouter les ingrédients comme les 
olives tranchées, les tomates fraîches hachées, le poivron, et éventuellement du piment jalapeño et de la salsa. 
Enfin, saupoudrer de cheddar râpé. Cuire le tout au four pendant environ 10 minutes, ou jusqu'à ce que le fromage 
ait fondu et que le dessus soit légèrement doré. Le plat est prêt à être partagé une fois cuit.",
    response="Pour cuisiner un nachos garnis, commencez par préchauffer le four à 400 °F. Étendez ensuite les 
croustilles de maïs sur une plaque de cuisson recouverte de papier-parchemin. Ajoutez dessus les olives, les 
tomates fraîches hachées, le poivron, et si désiré, du piment jalapeño et de la salsa. Saupoudrez de cheddar râpé. 
Placez la plaque au four et faites cuire pendant environ 10 minutes, ou jusqu'à ce que le fromage soit fondu et 
légèrement doré. Servez chaud et partagez."
)

[
    {
        'prompt': None,
        'messages': [
            {
                'role': 'system',
                'content': 'Your input fields are:\n1. `context` (str): \n2. `question` (str):\nYour output fields 
are:\n1. `reasoning` (str): \n2. `response` (str):\nAll interactions will be structured in the following way, with 
the appropriate values filled in.\n\n[[ ## context ## ]]\n{context}\n\n[[ ## question ## ]]\n{question}\n\n[[ ## 
reasoning ## ]]\n{reasoning}\n\n[[ ## response ## ]]\n{response}\n\n[[ ## completed ## ]]\nIn adhering to this 
structure, your objective is: \n        Given the fields `context`, `question`, produce the fields `response`.'
            },
            {
                'role': 'user',
                'content': "[[ ## context ## ]]\n[1] «««\n    title=LIVRE DE RECETTES POUR ETUDIANTS PAUVRES > 
NACHOS GARNIS entrees (A PARTAGER) > PREPARATION | text=1. Préchauffer le four à 400 ° F\n    2. Sur une plaque de 
cuisson tapissée de papier-parchemin, étendre les croustilles de maïs.\n    3. Ajouter les olives, les tomates 
fraiches et le poivron. Si désiré, le piment jalapeño et la salsa.\n    4. Garnir de cheddar râpé.\n    5. Cuire 
pendant environ 10 minutes ou jusqu'à ce que le fromage ait fondu et qu'il soit légèrement doré.\n»»»\n[2] «««\n   
title=LIVRE DE RECETTES POUR ETUDIANTS PAUVRES > NACHOS GARNIS entrees (A PARTAGER) > INGREDIENTS | text=- 4 tasses
de croustilles de maïs\n    - 3 c. à table (45 ml) d'olives noires coupées en tranches\n    - 3 c. à table (45 ml) 
de tomates fraîches, épépinées et hachées (ou tomates en conserve)\n    - 1 tasse(250 ml) de cheddar râpé\n    - 
1/4 tasse (60 ml) de salsa (facultatif)\n    - 1 c. à table (15 ml) de piment jalapeño haché 
(facultatif)\n»»»\n\n[[ ## question ## ]]\nComment cuisiner un nachos garnis ?\n\nRespond with the corresponding 
output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## response ## ]]`, and then ending with 
the marker for `[[ ## completed ## ]]`."
            }
        ],
        'kwargs': {},
        'response': ModelResponse(
            id='chatcmpl-D5yxzwrWxkZliw15D3hAYSomwCM8h',
            created=1770318367,
            model='gpt-4.1-nano-2025-04-14',
            object='chat.completion',
            system_fingerprint='fp_70fc5e01a5',
            choices=[
                Choices(
                    finish_reason='stop',
                    index=0,
                    message=Message(
                        content="[[ ## reasoning ## ]]\nPour cuisiner un nachos garnis, il faut d'abord préchauffer
le four à 400 °F. Ensuite, étaler les croustilles de maïs sur une plaque de cuisson recouverte de papier-parchemin.
Ajouter les ingrédients comme les olives tranchées, les tomates fraîches hachées, le poivron, et éventuellement du 
piment jalapeño et de la salsa. Enfin, saupoudrer de cheddar râpé. Cuire le tout au four pendant environ 10 
minutes, ou jusqu'à ce que le fromage ait fondu et que le dessus soit légèrement doré. Le plat est prêt à être 
partagé une fois cuit.\n\n[[ ## response ## ]]\nPour cuisiner un nachos garnis, commencez par préchauffer le four à
400 °F. Étendez ensuite les croustilles de maïs sur une plaque de cuisson recouverte de papier-parchemin. Ajoutez 
dessus les olives, les tomates fraîches hachées, le poivron, et si désiré, du piment jalapeño et de la salsa. 
Saupoudrez de cheddar râpé. Placez la plaque au four et faites cuire pendant environ 10 minutes, ou jusqu'à ce que 
le fromage soit fondu et légèrement doré. Servez chaud et partagez.\n\n[[ ## completed ## ]]",
                        role='assistant',
                        tool_calls=None,
                        function_call=None,
                        provider_specific_fields={'refusal': None},
                        annotations=[]
                    ),
                    provider_specific_fields={}
                )
            ],
            usage=Usage(
                completion_tokens=280,
      





[2026-02-05T20:06:09.914950]

System message:

Your input fields are:
1. `context` (str): 
2. `question` (str):
Your output fields are:
1. `reasoning` (str): 
2. `response` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `context`, `question`, produce the fields `response`.


User message:

[[ ## context ## ]]
[1] «««
    title=LIVRE DE RECETTES POUR ETUDIANTS PAUVRES > NACHOS GARNIS entrees (A PARTAGER) > PREPARATION | text=1. Préchauffer le four à 400 ° F
    2. Sur une plaque de cuisson tapissée de papier-parchemin, étendre les croustilles de maïs.
    3. Ajouter les olives, les tomates fraiches et le poivron. Si désiré, le piment jalapeño et la salsa.
    4. Garnir de cheddar râpé.
    5. Cuire p